### 导入必要的库

In [42]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.optimizers import Adam

In [3]:
tf.__version__

'1.8.0'

In [4]:
from mnist import MNIST
data = MNIST(data_dir="data/MNIST/")

In [5]:
print("Size of:")
print("- Training-set:\t\t{}".format(data.num_train))
print("- Validation-set:\t{}".format(data.num_val))
print("- Test-set:\t\t{}".format(data.num_test))

Size of:
- Training-set:		55000
- Validation-set:	5000
- Test-set:		10000


In [6]:
# 数据的维度信息
# The number of pixels in each dimension of an image.
img_size = data.img_size

# The images are stored in one-dimensional arrays of this length.
img_size_flat = data.img_size_flat

# Tuple with height and width of images used to reshape arrays.
img_shape = data.img_shape

# Tuple with height, width and depth used to reshape arrays.
# This is used for reshaping in Keras.
img_shape_full = data.img_shape_full

# Number of classes, one class for each of 10 digits.
num_classes = data.num_classes

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = data.num_channels

### 用keras搭建模型

In [13]:
model = Sequential()
model.add(InputLayer(input_shape=(img_size_flat, )))
model.add(Reshape(img_shape_full))
model.add(Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), padding='SAME', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="SAME"))
model.add(Conv2D(filters=36, kernel_size=(5, 5), strides=(1, 1), padding='SAME', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='SAME'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               225920    
__________

In [18]:
# 创建优化器 编译模型
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
history = model.fit(
    x=data.x_train, 
    y=data.y_train, 
    batch_size=32, 
    verbose=1, 
    epochs=20, 
    validation_data=(data.x_val, data.y_val)
)

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 7s 121us/step - loss: 0.0232 - acc: 0.9931 - val_loss: 0.0349 - val_acc: 0.9902
Epoch 2/20
55000/55000 [==============================] - 7s 122us/step - loss: 0.0208 - acc: 0.9933 - val_loss: 0.0374 - val_acc: 0.9896
Epoch 3/20
55000/55000 [==============================] - 6s 111us/step - loss: 0.0191 - acc: 0.9937 - val_loss: 0.0376 - val_acc: 0.9894
Epoch 4/20
55000/55000 [==============================] - 6s 102us/step - loss: 0.0165 - acc: 0.9952 - val_loss: 0.0408 - val_acc: 0.9892
Epoch 5/20
55000/55000 [==============================] - 6s 102us/step - loss: 0.0147 - acc: 0.9954 - val_loss: 0.0382 - val_acc: 0.9910
Epoch 6/20
55000/55000 [==============================] - 6s 102us/step - loss: 0.0136 - acc: 0.9954 - val_loss: 0.0446 - val_acc: 0.9870
Epoch 7/20
55000/55000 [==============================] - 6s 102us/step - loss: 0.0117 - acc: 0.9964 - val_loss: 0.0374 - v

### Keras 函数式编程模型

In [30]:
inputs = Input(shape=(img_size_flat, ))
reshape_inputs = Reshape(img_shape_full)(inputs)

In [33]:
conv1 = Conv2D(
    filters=16, 
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='SAME',
    activation='relu'
)(reshape_inputs)

In [35]:
maxpool1 = MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2),
    padding='SAME'
)(conv1)

In [36]:
conv2 = Conv2D(
    filters=36,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding="SAME",
    activation='relu'
)(maxpool1)

In [37]:
maxpool2 = MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2),
    padding="SAME"
)(conv2)

In [38]:
flatten = Flatten()(maxpool2)

In [39]:
dense1 = Dense(128, activation='relu')(flatten)

In [40]:
output_layer = Dense(10, activation='softmax')(dense1)

In [43]:
model2 = Model(inputs=inputs, outputs=output_layer)

In [44]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 784)               0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1764)              0         
__________

In [45]:
# 创建优化器 编译模型
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(
    x=data.x_train, 
    y=data.y_train, 
    batch_size=32, 
    verbose=1, 
    epochs=20, 
    validation_data=(data.x_val, data.y_val)
)

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 4s 75us/step - loss: 0.2079 - acc: 0.9388 - val_loss: 0.0880 - val_acc: 0.9762
Epoch 2/20
55000/55000 [==============================] - 4s 70us/step - loss: 0.0839 - acc: 0.9749 - val_loss: 0.0615 - val_acc: 0.9836
Epoch 3/20
55000/55000 [==============================] - 4s 70us/step - loss: 0.0636 - acc: 0.9807 - val_loss: 0.0597 - val_acc: 0.9836
Epoch 4/20
55000/55000 [==============================] - 4s 70us/step - loss: 0.0518 - acc: 0.9841 - val_loss: 0.0503 - val_acc: 0.9838
Epoch 5/20
55000/55000 [==============================] - 4s 70us/step - loss: 0.0432 - acc: 0.9868 - val_loss: 0.0485 - val_acc: 0.9858
Epoch 6/20
55000/55000 [==============================] - 4s 70us/step - loss: 0.0379 - acc: 0.9883 - val_loss: 0.0417 - val_acc: 0.9884
Epoch 7/20
55000/55000 [==============================] - 4s 70us/step - loss: 0.0324 - acc: 0.9899 - val_loss: 0.0411 - val_acc: